## 本文档实现了一个，根据用户、商品属性预测单个商品是否购买的模型，参考了DIN，但使用了比较简单模型输入

202108

## pre

In [ ]:
!mkdir /content/drive/MyDrive/202108-bigdatacup2021

In [ ]:
%cd /content/drive/MyDrive/202108-bigdatacup2021

/content/drive/MyDrive/202108-bigdatacup2021


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [ ]:
class VanillaBaseModel(nn.Module):
    def __init__(self, 
                 num_items,
                 dim_item_emb=64,
                 dim_item_discrete_feature_emb=16,
                 dim_user_discrete_feature_emb=16,
                ):
        super().__init__()
        self.NUM_ITEM_DISCRETE_FEATURE = 3 + 1 # item_vec3 + location1
        self.NUM_ITEM_CONT_FEATURE = 2 + 1 # item_vec2 + price1
        self.NUM_USER_DISCRETE_FEATURE = 10
        self.dim_item_emb = dim_item_emb

        self.item_emb = nn.Embedding(num_items + 1, dim_item_emb) # 0表示没有记录，因此 num_items + 1
        
        # item discrete feature
        self.item_discrete_feature_emb_list = nn.ModuleList()
        num_unique_value_list = [4, 10, 2, 3]
        for i in range(self.NUM_ITEM_DISCRETE_FEATURE):
            num_unique_value = num_unique_value_list[i]
            self.item_discrete_feature_emb_list.append(
                nn.Embedding(num_unique_value, dim_item_discrete_feature_emb)
            )
        
        # user discrete feature
        self.user_discrete_feature_emb_list = nn.ModuleList()
        num_unique_value_list = [3, 1430, 20, 10, 198, 52, 3, 13, 2, 2347]
        for i in range(self.NUM_USER_DISCRETE_FEATURE):
            num_unique_value = num_unique_value_list[i]
            self.user_discrete_feature_emb_list.append(
                nn.Embedding(num_unique_value, dim_user_discrete_feature_emb)
            )

        # backbone
        self.backbone = nn.Sequential(
            nn.Linear(dim_item_emb + # user_click_history
                      self.NUM_ITEM_DISCRETE_FEATURE * dim_item_discrete_feature_emb + 
                      self.NUM_ITEM_CONT_FEATURE + 
                      self.NUM_USER_DISCRETE_FEATURE * dim_user_discrete_feature_emb +
                      dim_item_emb, 200), 
            nn.Dropout(0.1),
            nn.PReLU(),
            nn.LayerNorm(200),
            nn.Linear(200, 80),
            nn.PReLU(),
            nn.Dropout(0.1),
            nn.LayerNorm(80),
            nn.Linear(80, 1)
        )


    def forward(self,
                user_click_history,
                num_user_click_history,
                user_discrete_feature,
                item_id,
                item_discrete_feature,
                item_cont_feature
                ):
        """
        user_click_history: [N, 300], 最多有300个点击历史记录, 每个里面是itemid, 0表示没有记录
        num_user_click_history: [N, 1], 用户点击历史数量
        user_discrete_feature: [N, 10] 用户离散属性（已重映射）
        item_id: [N, 1], 商品id
        item_discrete_feature: [N, 3 + 1] 商品离散属性（已重映射） item_vec3 + location1
        item_cont_feature: [N, 2 + 1] 商品连续属性（已正则化） item_vec2 + price1
        """

        batch_size = user_click_history.size()[0]
        
        ## User Bahaviors 这里只用了item_emb，没有用这些商品的属性，跟DIN论文不太一样，先这么写试试
        # user click history emb
        tmp = self.item_emb(user_click_history) # [N, 300] -> [N, 300, dim_item_emb]
        user_click_history_emb = torch.zeros((batch_size, self.dim_item_emb))
        for i in range(batch_size):
            #print(num_user_click_history[i])
            aa = tmp[i, :num_user_click_history[i], :] # [N, D]
            #print(aa.shape)
            a = torch.mean(aa, dim=0) # [N, d] -> [1, d]
            #print(a.shape)
            #print(user_click_history_emb.shape)
            user_click_history_emb[i] = a

        ## User Profile Features
        # user discrete feature, 10 features
        tmp = []
        for i in range(self.NUM_USER_DISCRETE_FEATURE):
            tmp.append(
                self.user_discrete_feature_emb_list[i](user_discrete_feature[:, i]) # [N, dim_user_discrete_feature_emb]
            )
        user_discrete_feature_emb = torch.cat(tmp, dim=1)

        ## Item
        # item discrete feature, 3 features
        tmp = []
        for i in range(self.NUM_ITEM_DISCRETE_FEATURE):
            # print(i)
            # print(item_discrete_feature[:, i])
            tmp.append(
                self.item_discrete_feature_emb_list[i](item_discrete_feature[:, i]) # [N, dim_user_discrete_feature_emb]
            )
        item_discrete_feature_emb = torch.cat(tmp, dim=1)
        # item emb
        item_emb = self.item_emb(item_id)
        item_emb = torch.squeeze(item_emb)

        ## all emb
        #print(user_click_history_emb.size())
        #print(user_discrete_feature_emb.size())
        #print(item_discrete_feature_emb.size())
        #print(item_cont_feature.size())
        #print(item_emb.size())

        all_emb = torch.cat([user_click_history_emb, 
                             user_discrete_feature_emb,
                             item_discrete_feature_emb,
                             item_cont_feature,
                             item_emb,
                            ], dim=1) # [N, D]
        
        out = self.backbone(all_emb) # [N, 1]
        return out
                

In [ ]:
m = VanillaBaseModel(
    num_items=381,
    dim_item_emb=64,
    dim_item_discrete_feature_emb=16,
    dim_user_discrete_feature_emb=16,
)

In [ ]:
# user_click_history: [N, 300], 最多有300个点击历史记录, 每个里面是itemid, 0表示没有记录
# num_user_click_history: [N, 1], 用户点击历史数量
# user_discrete_feature: [N, 10] 用户离散属性（已重映射）
# item_id: [N, 1], 商品id
# item_discrete_feature: [N, 3 + 1] 商品离散属性（已重映射）
# item_cont_feature: [N, 2 + 1] 商品连续属性（已正则化）

B = 3
m(
    user_click_history=torch.ones([B, 300], dtype=torch.int32),
    num_user_click_history=torch.ones([B, 1], dtype=torch.int32) * 10,
    user_discrete_feature=torch.ones([B, 10], dtype=torch.int32),
    item_id=torch.ones([B, 1], dtype=torch.int32),
    item_discrete_feature=torch.ones([B, 4], dtype=torch.int32),
    item_cont_feature=torch.randn([B, 3]),
)

tensor([[-1.0110],
        [-1.3368],
        [-1.4544]], grad_fn=<AddmmBackward>)

## dataloader

In [ ]:
import pandas as pd
from sklearn.utils import shuffle
data_path='/content/drive/MyDrive/202108-bigdatacup2021/data/'
df_item_info = pd.read_csv(f'{data_path}/item_info.csv', sep=' ')

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.utils import shuffle

In [ ]:
## 获取 user portrait 的映射，因为
data_path='/content/drive/MyDrive/202108-bigdatacup2021/data/'
# portraitidx_to_idx_dict_list: list of 10 dict, int:int

portraitidx_to_idx_dict_list = []
for i in range(10):
    portraitidx_to_idx_dict_list.append(dict())
acculumated_idx = [0] * 10


df_train = pd.read_csv(f'{data_path}/trainset.csv', sep=' ')
for i in tqdm(range(df_train.shape[0])):
    user_portrait = [int(s) for s in df_train.at[i, 'user_protrait'].split(',')]
    for idx, u in enumerate(user_portrait):
        if portraitidx_to_idx_dict_list[idx].get(u, -1) == -1:
            portraitidx_to_idx_dict_list[idx][u] = acculumated_idx[idx]
            acculumated_idx[idx] += 1

df_test1 = pd.read_csv(f'{data_path}/track1_testset.csv', sep=' ')
for i in tqdm(range(df_test1.shape[0])):
    user_portrait = [int(s) for s in df_test1.at[i, 'user_protrait'].split(',')]
    for idx, u in enumerate(user_portrait):
        if portraitidx_to_idx_dict_list[idx].get(u, -1) == -1:
            portraitidx_to_idx_dict_list[idx][u] = acculumated_idx[idx]
            acculumated_idx[idx] += 1

df_test2 = pd.read_csv(f'{data_path}/track2_testset.csv', sep=' ')
for i in tqdm(range(df_test2.shape[0])):
    user_portrait = [int(s) for s in df_test2.at[i, 'user_protrait'].split(',')]
    for idx, u in enumerate(user_portrait):
        if portraitidx_to_idx_dict_list[idx].get(u, -1) == -1:
            portraitidx_to_idx_dict_list[idx][u] = acculumated_idx[idx]
            acculumated_idx[idx] += 1

acculumated_idx

100%|██████████| 206096/206096 [00:02<00:00, 72320.06it/s]


[3, 1430, 20, 10, 198, 52, 3, 13, 2, 2347]

In [ ]:
def load_data(data_path='/content/drive/MyDrive/202108-bigdatacup2021/data/'):
    # item info
    df_item_info = pd.read_csv(f'{data_path}/item_info.csv', sep=' ')
    item_info_dict = {}
    for i in tqdm(range(df_item_info.shape[0])):
        item_id = df_item_info.at[i, 'item_id'] 

        item_discrete = df_item_info.at[i, 'item_vec'].split(',')[:3]
        item_cont = df_item_info.at[i, 'item_vec'].split(',')[-2:]
        price = df_item_info.at[i, 'price'] / 3000
        loc = df_item_info.at[i, 'location'] - 1 # 0~2

        item_cont.append(price) # 2 + 1
        item_discrete.append(loc) # 3 + 1

        item_cont = [float(it) for it in item_cont]
        item_discrete = [int(it) for it in item_discrete]
        item_discrete[0] = item_discrete[0] - 1 # 1~4 -> 0~3
        item_discrete[2] = item_discrete[2] - 1 # 1~2 -> 0~1

        item_info_dict[int(item_id)] = {
            'cont': np.array(item_cont, dtype=np.float64),
            'discrete': np.array(item_discrete, dtype=np.int64),
        }

    # trainset
    train_samples = []
    val_samples = []
    df_train = pd.read_csv(f'{data_path}/trainset.csv', sep=' ')

    # shuffle
    df_train = shuffle(df_train, random_state=2333).reset_index()
    total_num = int(df_train.shape[0])
    num_train = int(total_num * 0.95)
    num_val = total_num - num_train

    for i in tqdm(range(total_num)):
        if df_train.at[i, 'user_click_history'] == '0:0':
            user_click_list = [0]
        else:
            user_click_list = df_train.at[i, 'user_click_history'].split(',')
            user_click_list = [int(sample.split(':')[0]) for sample in user_click_list]
        num_user_click_history = len(user_click_list)
        tmp = np.zeros(400, dtype=np.int64)
        tmp[:len(user_click_list)] = user_click_list
        user_click_list = tmp
        
        exposed_items = [int(s) for s in df_train.at[i, 'exposed_items'].split(',')]
        labels = [int(s) for s in df_train.at[i, 'labels'].split(',')]

        user_portrait = [int(s) for s in df_train.at[i, 'user_protrait'].split(',')]
        # portraitidx_to_idx_dict_list: list of 10 dict, int:int
        for j in range(10):
            user_portrait[j] = portraitidx_to_idx_dict_list[j][user_portrait[j]]
        for k in range(9):
            one_sample = {
                'user_click_list': user_click_list,
                'num_user_click_history': num_user_click_history,
                'user_portrait': np.array(user_portrait, dtype=np.int64),
                'item_id': exposed_items[k],
                'label': labels[k]
            }
            if i < num_train:
                train_samples.append(one_sample)
            else:
                val_samples.append(one_sample)
    return item_info_dict, train_samples, val_samples


class BigDataCupDataset(torch.utils.data.Dataset):
    def __init__(self, 
                 item_info_dict,
                 database
                ):
        super().__init__()
        self.item_info_dict = item_info_dict
        self.database = database

    def __len__(self, ):
        return len(self.database)

    def __getitem__(self, idx):
        one_sample = self.database[idx]
        user_click_history = one_sample['user_click_list']
        num_user_click_history = one_sample['num_user_click_history']
        user_discrete_feature = one_sample['user_portrait']
        item_id = one_sample['item_id']
        item_discrete_feature = self.item_info_dict[item_id]['discrete']
        item_cont_feature = self.item_info_dict[item_id]['cont']
        label = one_sample['label']

        # print(num_user_click_history)

        user_click_history = torch.IntTensor(user_click_history)
        num_user_click_history = torch.IntTensor([num_user_click_history])
        user_discrete_feature = torch.IntTensor(user_discrete_feature)
        item_id = torch.IntTensor([item_id])
        item_discrete_feature = torch.IntTensor(item_discrete_feature)
        item_cont_feature = torch.FloatTensor(item_cont_feature)
        label = torch.IntTensor([label])

        # print(num_user_click_history)

        return user_click_history, num_user_click_history, user_discrete_feature, \
               item_id, item_discrete_feature, item_cont_feature, label

In [ ]:
def load_test_data(data_path='/content/drive/MyDrive/202108-bigdatacup2021/data/', filename='track1_testset.csv'):
    # item info
    df_item_info = pd.read_csv(f'{data_path}/item_info.csv', sep=' ')
    item_info_dict = {}
    for i in tqdm(range(df_item_info.shape[0])):
        item_id = df_item_info.at[i, 'item_id'] 

        item_discrete = df_item_info.at[i, 'item_vec'].split(',')[:3]
        item_cont = df_item_info.at[i, 'item_vec'].split(',')[-2:]
        price = df_item_info.at[i, 'price'] / 3000
        loc = df_item_info.at[i, 'location'] - 1 # 0~2

        item_cont.append(price) # 2 + 1
        item_discrete.append(loc) # 3 + 1

        item_cont = [float(it) for it in item_cont]
        item_discrete = [int(it) for it in item_discrete]
        item_discrete[0] = item_discrete[0] - 1 # 1~4 -> 0~3
        item_discrete[2] = item_discrete[2] - 1 # 1~2 -> 0~1

        item_info_dict[int(item_id)] = {
            'cont': np.array(item_cont, dtype=np.float64),
            'discrete': np.array(item_discrete, dtype=np.int64),
        }

    # testset
    test_samples = []
    df_test = pd.read_csv(f'{data_path}/{filename}', sep=' ')

    # shuffle
    total_num = int(df_test.shape[0])

    for i in tqdm(range(total_num)):
        if df_test.at[i, 'user_click_history'] == '0:0':
            user_click_list = [0]
        else:
            user_click_list = df_test.at[i, 'user_click_history'].split(',')
            user_click_list = [int(sample.split(':')[0]) for sample in user_click_list]
        num_user_click_history = len(user_click_list)
        tmp = np.zeros(400, dtype=np.int64)
        tmp[:len(user_click_list)] = user_click_list
        user_click_list = tmp
        
        exposed_items = [int(s) for s in df_test.at[i, 'exposed_items'].split(',')]
        labels = [int(s) for s in df_test.at[i, 'labels'].split(',')]

        user_portrait = [int(s) for s in df_test.at[i, 'user_protrait'].split(',')]
        # portraitidx_to_idx_dict_list: list of 10 dict, int:int
        for j in range(10):
            user_portrait[j] = portraitidx_to_idx_dict_list[j][user_portrait[j]]
        for k in range(9):
            one_sample = {
                'user_click_list': user_click_list,
                'num_user_click_history': num_user_click_history,
                'user_portrait': np.array(user_portrait, dtype=np.int64),
                'item_id': exposed_items[k],
            }
            test_samples.append(one_sample)
    return item_info_dict, test_samples


class BigDataCupTestDataset(torch.utils.data.Dataset):
    def __init__(self, 
                 item_info_dict,
                 database
                ):
        super().__init__()
        self.item_info_dict = item_info_dict
        self.database = database

    def __len__(self, ):
        return len(self.database)

    def __getitem__(self, idx):
        one_sample = self.database[idx]
        user_click_history = one_sample['user_click_list']
        num_user_click_history = one_sample['num_user_click_history']
        user_discrete_feature = one_sample['user_portrait']
        item_id = one_sample['item_id']
        item_discrete_feature = self.item_info_dict[item_id]['discrete']
        item_cont_feature = self.item_info_dict[item_id]['cont']

        user_click_history = torch.IntTensor(user_click_history)
        num_user_click_history = torch.IntTensor([num_user_click_history])
        user_discrete_feature = torch.IntTensor(user_discrete_feature)
        item_id = torch.IntTensor([item_id])
        item_discrete_feature = torch.IntTensor(item_discrete_feature)
        item_cont_feature = torch.FloatTensor(item_cont_feature)

        return user_click_history, num_user_click_history, user_discrete_feature, \
               item_id, item_discrete_feature, item_cont_feature

## training

In [ ]:
item_info_dict, train_samples, val_samples = load_data()

train_ds = BigDataCupDataset(item_info_dict, train_samples)
train_dl = torch.utils.data.DataLoader(dataset=train_ds, batch_size=32, shuffle=True)

val_ds = BigDataCupDataset(item_info_dict, val_samples)
val_dl = torch.utils.data.DataLoader(dataset=val_ds, batch_size=9, shuffle=False)

100%|██████████| 260087/260087 [00:31<00:00, 8232.60it/s]


In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum / y_test.shape[0]
    return acc


## below: only applicable for batch_size==9 in validation
def real_acc_calc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum / y_test.shape[0]
    if acc != 1.0:
        return 0
    else:
        return 1

def real_acc_rule_calc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    
    cum_sum = 0
    for j in range(9):
        if y_pred_tag[j][0] == 1:
            cum_sum += 1
        if j == 2 and cum_sum != 3:
            y_pred_tag[3:] = 0
            break
        if j == 5 and cum_sum != 6:
            y_pred_tag[6:] = 0
            break
    
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum / y_test.shape[0]
    if acc != 1.0:
        return 0
    else:
        return 1

def real_acc_rule2_calc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    
    cum_sum = 0
    for j in range(9):
        k = 8 - j
        if k >= 6 and y_pred_tag[k][0] == 1:
            y_pred_tag[:6] = 1
        if k >= 3 and y_pred_tag[k][0] == 1:
            y_pred_tag[:3] = 1
    
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum / y_test.shape[0]
    if acc != 1.0:
        return 0
    else:
        return 1

In [ ]:
model = VanillaBaseModel(
    num_items=381,
    dim_item_emb=16,
    dim_item_discrete_feature_emb=16,
    dim_user_discrete_feature_emb=16,
)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


NUM_EPOCH = 2
for epoch_idx in range(NUM_EPOCH):  # loop over the dataset multiple times

    running_loss = 0.0
    train_cnt = 0
    train_acc_sum = 0
    
    for i, data in enumerate(train_dl, 0):
        model.train()

        # get the inputs; data is a list of [inputs, labels]
        user_click_history, num_user_click_history, user_discrete_feature, \
               item_id, item_discrete_feature, item_cont_feature, label = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(user_click_history, num_user_click_history, user_discrete_feature, \
               item_id, item_discrete_feature, item_cont_feature)

        loss = criterion(outputs, label.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        acc = binary_acc(outputs, label)
        train_acc_sum += acc
        train_cnt += 1

        if i % 1000 == 1:    # print every 2000 mini-batches
            print('----- TRAIN -----')
            print('[%d, %5d] loss: %.3f' % (epoch_idx + 1, i + 1, running_loss))
            print('- acc:', train_acc_sum / train_cnt, flush=True)

            running_loss = 0.0
            train_cnt = 0
            train_acc_sum = 0
            # print(outputs, label)

            ## val
            model.eval()
            cnt = 0
            acc_sum = 0
            real_acc_sum = 0
            real_rule_acc_sum = 0
            real_rule2_acc_sum = 0
            
            for _, val_data in tqdm(enumerate(val_dl, 0)):
                user_click_history, num_user_click_history, user_discrete_feature, \
                    item_id, item_discrete_feature, item_cont_feature, label = val_data
                outputs = model(user_click_history, num_user_click_history, user_discrete_feature, \
                    item_id, item_discrete_feature, item_cont_feature)
                acc = binary_acc(outputs, label)
                real_acc = real_acc_calc(outputs, label)
                real_rule_acc = real_acc_rule_calc(outputs, label)
                real_rule2_acc = real_acc_rule2_calc(outputs, label)
                
                acc_sum += acc
                real_acc_sum += real_acc
                real_rule_acc_sum += real_rule_acc
                real_rule2_acc_sum += real_rule2_acc

                cnt += 1
            print('----- VAL -----')
            print('- acc:', acc_sum / cnt)
            print('- real acc:', real_acc_sum / cnt)
            print('- real rule acc:', real_rule_acc_sum / cnt)
            print('- real rule2 acc:', real_rule2_acc_sum / cnt)

print('Finished Training')

----- TRAIN -----
[1,     2] loss: 1.438
- acc: tensor(0.5156)


13005it [00:36, 359.67it/s]


----- VAL -----
- acc: tensor(0.5883)
- real acc: 0.08227604767397155
- real rule acc: 0.0946559015763168
- real rule2 acc: 0.12856593617839293
----- TRAIN -----
[1,  1002] loss: 528.724
- acc: tensor(0.7445)


13005it [00:35, 362.85it/s]


----- VAL -----
- acc: tensor(0.7494)
- real acc: 0.20499807766243752
- real rule acc: 0.21491733948481354
- real rule2 acc: 0.24159938485198001
----- TRAIN -----
[1,  2002] loss: 519.285
- acc: tensor(0.7462)


...

13005it [00:36, 352.42it/s]


----- VAL -----
- acc: tensor(0.7831)
- real acc: 0.2612072279892349
- real rule acc: 0.2711264898116109
- real rule2 acc: 0.2895040369088812
----- TRAIN -----
[2,  5002] loss: 457.637
- acc: tensor(0.7881)


13005it [00:37, 351.08it/s]


----- VAL -----
- acc: tensor(0.7828)
- real acc: 0.2668973471741638
- real rule acc: 0.2768935024990388
- real rule2 acc: 0.29219530949634753
----- TRAIN -----
[2,  6002] loss: 462.897
- acc: tensor(0.7829)


6960it [00:19, 358.05it/s]

In [ ]:
torch.save(model, 'vanilla_model_epoch1.2.pth')

In [ ]:
torch.save(model, 'vanilla_layernorm_model_epoch1.2.pth')

## test script

In [ ]:
item_info_dict, test_samples = load_test_data(data_path='/content/drive/MyDrive/202108-bigdatacup2021/data/', filename='track1_testset.csv')

test_ds = BigDataCupTestDataset(item_info_dict, test_samples)
test_dl = torch.utils.data.DataLoader(dataset=test_ds, batch_size=9, shuffle=False)

100%|██████████| 206254/206254 [00:18<00:00, 11116.89it/s]


In [ ]:
model = model.eval()

fp = open('vanilla_model_epoch1.2_track1.csv', 'w')
print('id,category', file=fp)

for i, data in tqdm(enumerate(test_dl, 0)):
    user_click_history, num_user_click_history, user_discrete_feature, \
            item_id, item_discrete_feature, item_cont_feature = data

    # forward + backward + optimize
    outputs = model(user_click_history, num_user_click_history, user_discrete_feature, \
            item_id, item_discrete_feature, item_cont_feature)
    
    y_pred_tag = torch.round(torch.sigmoid(outputs))

    ## rule1
    # cum_sum = 0
    # for j in range(9):
    #     if y_pred_tag[j][0] == 1:
    #         cum_sum += 1
    #     if j == 2 and cum_sum != 3:
    #         y_pred_tag[3:] = 0
    #         break
    #     if j == 5 and cum_sum != 6:
    #         y_pred_tag[6:] = 0
    #         break

    ## rule2
    cum_sum = 0
    for j in range(9):
        k = 8 - j
        if k >= 6 and y_pred_tag[k][0] == 1:
            y_pred_tag[:6] = 1
        if k >= 3 and y_pred_tag[k][0] == 1:
            y_pred_tag[:3] = 1
    
    y_pred_tag = list(y_pred_tag.detach().numpy()[:, 0].astype(np.int32))
    y_pred_tag = [str(a) for a in y_pred_tag]
    p = ' '.join(y_pred_tag)
    print(f'{i+1},{p}', file=fp)
    # break

fp.close()

206254it [05:33, 618.80it/s]
